## Optimization-Based Identification Methods for General Input-Output Model
This notebook demonstrates the use of optimization-based identification methods for general input-output models.

In [1]:
# Import required libraries
import numpy as np
from control import frequency_response
from control.timeresp import step_response
from sklearn.metrics import explained_variance_score, root_mean_squared_error
from sklearn.pipeline import Pipeline

from sippy_unipi.datasets import load_sample_siso
from sippy_unipi.io import ARARMAX, ARARX, ARMA, BJ, GEN, OE
from sippy_unipi.plot import (
    plot_bode,
    plot_response,
    plot_responses,
)
from sippy_unipi.preprocessing import StandardScaler

np.random.seed(0)
ylegends = ["System", "ARMA", "ARARX", "ARARMAX", "OE", "BJ", "GEN"]

# Enable automatic reloading of modules when they change
%load_ext autoreload
%autoreload 2

In [ ]:
# Load sample data
n_samples = 401
ts = 1.0
time, Ysim, Usim, g_sys, Yerr, Uerr, h_sys, Ytot, Utot = load_sample_siso(
    n_samples, ts, seed=0
)

# Plot input and output responses
fig = plot_responses(
    time,
    [Usim, Uerr, Utot],
    [Ysim, Yerr, Ytot],
    ["u", "e", ["u", "e"]],
)

In [3]:
na = 2
nb = 3
nc = 2
nd = 3
nf = 4
theta = 11
stab_cons = False
method = "opt"
models = [
    ARMA(na, nc, theta, stab_cons=stab_cons, method=method),
    ARARX(na, nb, nd, theta, stab_cons=stab_cons, method=method),
    ARARMAX(na, nb, nc, nd, theta, stab_cons=stab_cons, method=method),
    OE(nb, nf, theta, stab_cons=stab_cons, method=method),
    BJ(nb, nc, nd, nf, theta, stab_cons=stab_cons, method=method),
    GEN(na, nb, nc, nd, nf, theta, stab_cons=stab_cons, method=method),
]

In [ ]:
ys = [Ytot]
scores = {"rmse": [], "ev": []}
fitted_models = []
for model in models:
    sys = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("model", model),
        ]
    )
    sys.fit(Usim.reshape(-1, 1), Ytot.reshape(-1, 1))
    fitted_models.append(sys)
    Y_pred = sys.predict(Usim.reshape(-1, 1), safe=True)
    scores["rmse"].append(root_mean_squared_error(Ysim, Y_pred))
    scores["ev"].append(explained_variance_score(Ysim, Y_pred))
    ys.append(Y_pred)

In [ ]:
# Plot consistency of identified systems
fig = plot_response(
    time,
    ys,
    Usim,
    legends=[
        ["Original"] + [model.__class__.__name__ for model in models],
        ["U"],
    ],
    titles=[
        "Output (identification data)",
        "Input, identification data (Switch probability=0.08)",
    ],
)

In [6]:
# Validation of identified systems
switch_probability = 0.07
input_range = (0.5, 1.5)
noise_variance = 0.01
n_samples = 401
ts = 1.0
time, Ysimval, Usimval, g_sys, Yerrval, Uerrval, h_sys, Yval, Uval = (
    load_sample_siso(
        n_samples,
        ts,
        input_range=input_range,
        switch_probability=switch_probability,
        noise_variance=noise_variance,
        seed=0,
    )
)

ys = [sys.predict(Uval, safe=True) for sys in fitted_models]

In [ ]:
# Plot validation results
fig = plot_response(
    time,
    ys,
    Usim,
    legends=[ylegends, ["U"]],
    titles=[
        "Output (identification data)",
        "Input, identification data (Switch probability=0.07)",
    ],
)

In [ ]:
# Print scores in a formatted table
print("Model Performance Metrics:")
print("-" * 50)
print(f"{'Model':<20} {'RMSE':<10} {'Explained Variance':<20}")
print("-" * 50)

for i, model_name in enumerate(ylegends):
    if i == 0:  # Skip the first one which is the actual data
        continue
    rmse_value = scores["rmse"][i - 1]
    ev_value = scores["ev"][i - 1]
    print(f"{model_name:<20} {rmse_value:<10.4f} {ev_value:<20.4f}")
print("-" * 50)


In [ ]:
# Step tests
u = np.ones_like(time)
u[0] = 0
W_V = np.logspace(-3, 4, num=701)
for tf in ["G_", "H_"]:
    syss_tfs = [
        locals()[f"{tf.lower()}sys"],
        *[getattr(sys.steps[-1][1], tf) for sys in fitted_models],
    ]
    mags, fis, oms = zip(*[frequency_response(sys, W_V) for sys in syss_tfs])

    fig = plot_bode(
        oms[0],
        mags,
        fis,
        ylegends,
    )

    _, ys = zip(
        *[step_response(sys, time, transpose=True) for sys in syss_tfs]
    )

    fig = plot_response(
        time,
        ys,
        u,
        legends=[ylegends, ["U"]],
        titles=["Step Response G(z)", None],
    )